# CRNP field calibration of N0 device-specific parameter.

## $N_0$ Parameter Calibration

The calibration of a Cosmic-Ray Neutron Probe (CRNP) is an essential step to ensure accurate soil moisture measurements. The CRNP operates by counting fast neutrons produced from cosmic rays, which are predominantly moderated by water molecules in the soil. The parameter $N_0$ is a device-specific constant that signifies the neutron count rate under zero soil moisture conditions. 

$\theta(N) =\frac{a_0}{(\frac{N}{N_0}) - a_1} - a_2 $ (Desilets et al., 2010).

## Determining field soil moisture

### Soil sampling layout

The soil sampling layout is a critical component of the calibration process. It consists of three concentric layers of sampling points from the station location. This strategic arrangement ensures a comprehensive coverage of the area around the station.

<img src="../../../img/layout.png" style="max-width:500px">

Figure 1. Proposed layout for doing CRNP device field calibrations.


- **5 m:** At this closest proximity to the station, six sampling points are located equidistantly around the station. These points provide local soil data that is most immediately influenced by the station.
- **50 m:** This intermediate layer consists of four sampling points positioned at a distance of 50 meters from the central station. These points provide a broader perspective of the soil moisture content in the area.
- **100 m:** The outermost layer, situated at a distance of 100 meters from the central station, consists of four sampling points. This layer provides soil data from the furthest distance from the station within the sampling layout, offering a wider context of the soil moisture content in the area.

Each soil sample is then split into four depth segments: 0-5 cm, 5-10 cm, 10-25 cm, and 25-40 cm. This depth segmentation allows for a more detailed analysis of the soil moisture content at different soil depths, providing a more nuanced understanding of the soil's moisture profile.

<img src="../../../img/core.png" style="max-height:500px">

Figure 2. Proposed layout for spliting each of the soil cores by depth.

### Template for data collection

[Download the following template](https://github.com/soilwater/crnpy/blob/main/docs/examples/calibration/template.xlsx) spreadsheet for collecting soil samples data:

In [1]:
import pandas as pd
import numpy as np
from crnpy import crnpy

pd.read_excel("template.xlsx").replace(np.nan,'',regex=False).head()

,field,date,core_number,distance_from_station,latitude,longitude,top_depth,bottom_depth,core_diameter,wet_mass_with_bag,...,can_number,mass_empty_can,wet_mass_with_can,dry_mass_with_can,mass_water,theta_g,volume,bulk_density,theta_v,Observation
0,,,1,5,,,0,5,,,...,,,,,,,,,,
1,,,1,5,,,5,10,,,...,,,,,,,,,,
2,,,1,5,,,10,25,,,...,,,,,,,,,,
3,,,1,5,,,25,40,,,...,,,,,,,,,,
4,,,2,5,,,0,5,,,...,,,,,,,,,,


### Sample processing

The samples should be weighed as promptly as possible, either in the field or in the laboratory, to ensure accuracy and reliability of data. In the provided template the colums `wet_mass_with_bag	mass_empty_bag,mass_empty_can,wet_mass_with_can,` should be filled. After drying the samples at 105 °C degrees until constant weight is reached the column `dry_mass_with_can` needs to be filled.

At this point all the other columns can be filled with the calcualtions for obtaining gravimetric water content ($\theta_g$), bulk density ($\rho_\beta$) and the volumetric water content ($\theta_v$). See the details of this calculation on the [filled example](https://github.com/soilwater/crnpy/blob/main/docs/examples/calibration/soil_data.xlsx). 

### Field average

Load the soil samples data using pandas, and using the function `nrad_weight()` the weights corresponding to each soil sample will be computed considering air-humidity, sample depth, distance from station and bulk density.



In [2]:
df_soil = pd.read_excel("soil_data.xlsx")
df_soil.head()

,field,date,core_number,distance_from_station,latitude,longitude,top_depth,bottom_depth,core_diameter,wet_mass_with_bag,...,can_number,mass_empty_can,wet_mass_with_can,dry_mass_with_can,mass_water,theta_g,volume,bulk_density,theta_v,Observation
0,Flickner,2021-10-22,1,5,N38.23459,W97.57101,0,5,30.493333,45.31,...,1,52.10,92.03,85.31,6.72,0.202349,36.514864,0.909493,0.184403,NaN
1,Flickner,2021-10-22,1,5,N38.23459,W97.57101,5,10,30.493333,69.53,...,2,51.85,115.97,103.85,12.12,0.233077,36.514864,1.424078,0.332585,NaN
2,Flickner,2021-10-22,1,5,N38.23459,W97.57101,10,25,30.493333,214.90,...,3,51.56,260.97,219.77,41.20,0.244932,109.544592,1.535539,0.376856,NaN
3,Flickner,2021-10-22,1,5,N38.23459,W97.57101,25,40,30.493333,217.52,...,4,52.35,264.46,222.23,42.23,0.248587,109.544592,1.550784,0.386278,NaN
4,Flickner,2021-10-22,2,5,N38.23464,W97.57101,0,5,30.493333,45.92,...,5,52.15,92.59,85.93,6.66,0.197158,36.514864,0.925103,0.182757,NaN


Loading station data it is necessary to calculate the absolute humidty from temperature an relative humidity data.

In [3]:
df_station = pd.read_excel("station_data.xlsx", skiprows=[0,2,3])
df_station['abs_h'] = crnpy.estimate_abs_humidity(df_station['relative_humidity_Avg'], df_station['air_temperature_Avg'])
nrad_weights = crnpy.nrad_weight(df_station['abs_h'].mean(), df_soil['theta_v'], df_soil['distance_from_station'], (df_soil['bottom_depth']+df_soil['top_depth'])/2, rhob=df_soil['bulk_density'])

field_theta_v = np.sum(df_soil['theta_v']*nrad_weights)
field_bulk_density = np.sum(df_soil['bulk_density']*nrad_weights)
print(f"Field Volumetric Water content: {round(field_theta_v,3)}")

Field Volumetric Water content: 0.256


## Neutron count processing

Following the example for processing CRNP data from [RDT detectors](../../stationary/example_RDT_station/) neutron counts recorded while the field sampling was done need to be corrected.

[This dataset](https://github.com/soilwater/crnpy/blob/main/docs/examples/calibration/soil_data.xlsx) contains the raw counts and atmospheric data recorded by the CRNP while the field sampling was done.


In [4]:

df = pd.read_excel("station_data.xlsx", skiprows=[0,2,3])
# Parse timestamps and set as index
df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
df.set_index(df['TIMESTAMP'], inplace=True)

df.head()

,TIMESTAMP,RECORD,station,farm,field,latitude,longitude,altitude,PTemp_Avg,counts_1_Tot,counts_2_Tot,vapor_pressure_Avg,barometric_pressure_Avg,relative_humidity_Avg,humidity_sensor_temperature_Avg,air_temperature_Avg
TIMESTAMP,,,,,,,,,,,,,,,,
2021-10-22 07:00:00,2021-10-22 07:00:00,704,KS003,Flickner,Rainfed South,38.23461,-97.57095,455,7.232,774,796,8.75,965,80.20,7.95,8.30
2021-10-22 08:00:00,2021-10-22 08:00:00,705,KS003,Flickner,Rainfed South,38.23461,-97.57095,455,10.090,753,860,9.13,965,79.00,9.52,9.10
2021-10-22 09:00:00,2021-10-22 09:00:00,706,KS003,Flickner,Rainfed South,38.23461,-97.57095,455,17.330,736,796,10.18,965,73.95,13.38,11.68
2021-10-22 10:00:00,2021-10-22 10:00:00,707,KS003,Flickner,Rainfed South,38.23461,-97.57095,455,22.370,744,832,10.98,964,64.33,17.40,15.10
2021-10-22 11:00:00,2021-10-22 11:00:00,708,KS003,Flickner,Rainfed South,38.23461,-97.57095,455,25.880,762,838,10.98,964,52.10,21.20,18.30


#### Computing total neutron counts



In [5]:
df['total_counts'] = crnpy.compute_total_raw_counts(df[['counts_1_Tot','counts_2_Tot']], nan_strategy='average')

#### Atmospheric correction

The atmospheric correction factors will correct neutron counts for atmospheric pressure, and absolute humidity changes using the method described in Zreda et al. (2012) and Anderson et al. (2017).

In [6]:
# Fill NaN values in atmospheric data
df[['pressure', 'RH', 'T']] = crnpy.fill_missing_atm(df[['barometric_pressure_Avg', 'relative_humidity_Avg', 'air_temperature_Avg']])
# Correct count by atmospheric variables and incoming flux
df['total_counts']=crnpy.fill_counts(df['total_counts'])
df['corrected']=crnpy.atm_correction(df['total_counts'], pressure=df['pressure'], humidity=df['RH'], temp=df['T'], Pref=976, Aref=0, L=130).dropna()

No count time columns provided. Using timestamp index to compute count time.
Using median count time as expected count time: 3600.0


C:\Users\jperaza\AppData\Local\anaconda3\envs\crnpy\lib\site-packages\crnpy\crnpy.py:362: UserWarning: Ignoring incoming neutron flux correction factor (using value fi=1)
  warnings.warn("Ignoring incoming neutron flux correction factor (using value fi=1)")


In [7]:
print(f"Mean corrected neutron count during sampling: {df['corrected'].mean()}")

Mean corrected neutron count during sampling: 1539.8181818181818


## Solving the equation

Previous steps estimated the a field volumetric water content of `0.256` and an average neutron count of `1720`. The function [scipy.optimize.curve_fit()](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) estiamtes the $N_0$ parameter using least squares method.

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def crnpy_desilets_model(counts, N0):
    return crnpy.counts_to_vwc(counts, N0, bulk_density=field_bulk_density, Wlat=0.03, Wsoc=0.01)

# Generate some synthetic data
x = df['corrected'].mean()
y = field_theta_v
# Perform curve fitting
popt, pcov = curve_fit(crnpy_desilets_model, [x], [y])
print(popt)

[2636.69127046]


C:\Users\jperaza\AppData\Local\anaconda3\envs\crnpy\lib\site-packages\scipy\optimize\_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


## References: 
Dong, J., & Ochsner, T. E. (2018). Soil texture often exerts a stronger influence than precipitation on mesoscale soil moisture patterns. Water Resources Research, 54(3), 2199-2211.

Patrignani, A., Ochsner, T. E., Montag, B., & Bellinger, S. (2021). A novel lithium foil cosmic-ray neutron detector for measuring field-scale soil moisture. Frontiers in Water, 3, 673185.